# Tutorial: Compressing Natural Language With an Autoregressive Machine Learning Model and `constriction`

- **Author:** Robert Bamler, University of Tuebingen
- **Initial Publication Date:** Jan 7, 2022

This is an interactive jupyter notebook.
You can read this notebook [online](https://github.com/bamler-lab/constriction/blob/main/examples/python/03-tutorial-autoregressive-nlp-compression.ipynb) but if you want to execute any code, we recommend to [download](https://github.com/bamler-lab/constriction/raw/main/examples/python/03-tutorial-autoregressive-nlp-compression.ipynb) it.

More examples, tutorials, and reference materials are available at <https://bamler-lab.github.io/constriction/>.

## Introduction

This notebook teaches you how to losslessly compress and decompress data with `constriction` using an autoregressive machine learning model.
We will use the simple character based recurrent neural network toy-model for natural language from the [Practical PyTorch Series](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb).
You won't need any fancy hardware (like a GPU) to go along with this tutorial since the model is so simplistic that it can be trained on a normal consumer PC in about 30 minutes.

The compression method we develop in this tutorial is for demonstration purpose only and not meant as a proposal for practical text compression.
While you will find that our method can compress text that is similar to the training data very well (with a 40% reduction in bitrate compared to `bzip2`), you'll also find that it will generalize poorly to other text forms and that compression and decompression are excruciatingly slow.
The poor generalization performance is not a fundamental issue of the presented approach; it is just a result of using a very simplistic model combined with a very small training set.
The runtime performance, too, would improve to *some* degree if we used a better suited model and if we ported the implementation to a compiled language and avoid permanent data-copying between Python, PyTorch, and `constriction`.
However, even so, autoregressive models tend to be quite runtime-inefficient in general due to poor parallelizability.
An alternative to autoregressive models for exploiting correlations in data compression is the so-called bits-back technique.
An example of bits-back coding with `constriction` is provided in [this problem set](https://robamler.github.io/teaching/compress22/problem-set-07.zip) (with [solutions](https://robamler.github.io/teaching/compress21/problem-set-07-solutions.zip)).

## Step 1: Prepare Your Environment

We'll use the PyTorch deep learning framework to train and evaluate our entropy model.
Follow the [installation instructions](https://pytorch.org/get-started/locally/) (you'll save a *lot* of download time and disk space if you install the CPU-only version).
Then restart your jupyter kernel and test if you can import PyTorch:

In [1]:
import torch

Next, make sure you have a recent enough version of `constriction` and a few other small Python packages:

In [ ]:
%pip install constriction~=0.3.5 tqdm~=4.62.3 unidecode~=1.3.2

**Restart your jupyter kernel once the above installation is done.**
Then, let's get started.

## Step 2: Get Some Training Data

We'll train our text model on the same [100,000 character Shakespeare sample](https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt) that was used in the [Practical PyTorch Series](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb).
Different from the Practical PyTorch Series, however, we'll split the data set into a training set and a test set so that we can test our compression method on text on which the model wasn't explicitly trained.

Start with downloading the full data set:

In [1]:
!wget -O shakespeare_all.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2022-01-08 21:49:29--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1,1M) [text/plain]
Saving to: ‘shakespeare_all.txt’

shakespeare_all.txt 100%[===================>]   1,06M  6,37MB/s    in 0,2s    

2022-01-08 21:49:29 (6,37 MB/s) - ‘shakespeare_all.txt’ saved [1115394/1115394]



Now, let's split the data set into, say, 90% training data and 10% test data by randomly assigning each line of the full data set to either one of those two subsets.
In a real scientific project, you should usually split into more than just two sets (e.g., add an additional "validation" set that you then use to tune model hyperparameters).
But we'll keep it simple for this tutorial.

In [2]:
import numpy as np

target_test_set_ratio = 0.1 # means that about 10% of the lines will end up in the test set.
train_count, test_count = 0, 0
rng = np.random.RandomState(830472) # (always set a random seed to make results reproducible)

with open("shakespeare_all.txt", "r") as in_file, \
     open("shakespeare_train.txt", "w") as train_file, \
     open("shakespeare_test.txt", "w") as test_file:
    for line in in_file.readlines():
        if line == "\n" or line == "":
            continue # Let's leave out empty lines
        if rng.uniform() < target_test_set_ratio:
            test_file.write(line)
            test_count += 1
        else:
            train_file.write(line)
            train_count += 1

total_count = train_count + test_count
print(f"Total number of non-empty lines in the data set: {total_count}")
print(f"File `shakespeare_train.txt` has {train_count} lines ({100 * train_count / total_count:.1f}%).")
print(f"File `shakespeare_test.txt` has {test_count} lines ({100 * test_count / total_count:.1f}%).")

Total number of non-empty lines in the data set: 32777
File `shakespeare_train.txt` has 29524 lines (90.1%).
File `shakespeare_test.txt` has 3253 lines (9.9%).


## Step 3: Train The Model

We'll use the toy model described in [this tutorial](https://github.com/spro/practical-pytorch/blob/master/char-rnn-generation/char-rnn-generation.ipynb).
Luckily, someone has already extracted the relevant code blocks from the tutorial and built a command line application around it, so we'll just use that.

Clone the repository into a subdirectory `char-rnn.pytorch` right next to this notebook:

In [3]:
!git clone https://github.com/spro/char-rnn.pytorch

Cloning into 'char-rnn.pytorch'...
remote: Enumerating objects: 54, done.
remote: Total 54 (delta 0), reused 0 (delta 0), pack-reused 54
Receiving objects: 100% (54/54), 11.79 KiB | 5.89 MiB/s, done.
Resolving deltas: 100% (30/30), done.


At the time of writing, the `char-rnn.pytorch` repository seems to have a small bug that will result in an exception being raised when training the model (a pull request that fixes it [exists](https://github.com/spro/char-rnn.pytorch/pull/10)).
But we can fix it easily.
Open the file `char-rnn.pytorch/train.py` in a text editor, look for the line in the `train` function that reads

```python
    return loss.data[0] / args.chunk_len
```

and replace "`[0]`" with "`.item()`" so that the line now reads:

```python
    return loss.data.item() / args.chunk_len
```

If you can't find the original line then the bug has probably been fixed in the meantime.

Now, start the training procedure.
**This will take about 30 minutes** but you'll only have to do it once since the trained model will be saved to a file.

In [4]:
!python char-rnn.pytorch/train.py shakespeare_train.txt

Training for 2000 epochs...
  5%|██                                       | 99/2000 [01:57<23:18,  1.36it/s] 58s (100 5%) 1.8286]
Whird soul the dight.
What cit sust that a fantelfore ain tres, saine's mess honous herward 'y borrast 

 10%|███▉                                    | 199/2000 [03:53<58:27,  1.95s/it] 54s (200 10%) 1.5965]
What gracious. When shall am intters?
GLOUCESTER:
So for his wing that worrick thee
To at heart, and h 

 15%|█████▉                                  | 299/2000 [05:48<21:45,  1.30it/s] 49s (300 15%) 1.5209]
Where is these bluest?
GLOUCESTER: he will of have mercight.
My honours must dedther shall smerd, free 

 20%|███████▉                                | 399/2000 [07:36<20:12,  1.32it/s] 37s (400 20%) 1.5081]
What is grace be such receeds are counseal do limptrance,
And make out the rebed you grant and in the  

 25%|█████████▉                              | 499/2000 [09:22<20:48,  1.20it/s] 23s (500 25%) 1.4705]
Which have subbood for a pursup,
There

The above training script should print a few lines of text after each completed 5% of training progress.
The generated text snippets are random samples from the trained model.
You should be able to observe that the quality of the sampled text should improve as training proceeds.

At the end of the training cycle, the generated text will not be perfect, but that's OK for the purpose of compression.
When we'll use the trained model to compress some new text below, we won't be blindly following the model's predictions as in the randomly generated text here.
Instead, we'll compare the model's predictions with the actual text that we want to compress, and we'll then essentially encode the difference.
Thus, model predictions don't need to be perfect; as long as they're better than a completely random (uniform) guess, they will allow us to reduce the bitrate.

## Step 4: While the Model is Being Trained ...

While you wait for the model to be trained, let's start thinking about how we'll use the trained model for our compression method.

### Overall Encoder/Decoder Design

The model is an autoregressive model that processes one character after the other.
Each step takes the previous character as input, updates some internal hidden state, and outputs a probability distribution over all possible values for the next character.

This autoregressive model architecture pretty much dictates how our compression method must operate:

- The *encoder* reads the message that we want to compress character by character and updates the autoregressive model with each step.
  It uses the output of the model at each step to define an entropy model for encoding the next character.
- The *decoder* decodes one character at a time and uses it to perform the exact same model updates as the encoder did.
  The entropy model used for each decoding step is defined by the model output from the previous step.

The very first model update on both encoder and decoder side is a bit subtle because we don't yet have a "previous" character to provide as input for the model update here.
We'll just make up a fake character that we pretend exists before the start of the actual message, and we'll inject this fake character at the very beginning to both the encoder and the decoder.
Let's use the newline character `"\n"` for this purpose since this seems like a character that could indeed naturally precede the beginning of any message.

Since encoding and decoding iterate over the characters in the same order, we'll need to use an entropy coder with *queue* semantics (i.e., "first in first out").
The `constriction` library provides two entropy coders with queue semantics: [Range Coding](https://bamler-lab.github.io/constriction/apidoc/python/stream/queue.html) and [Huffman Coding](https://bamler-lab.github.io/constriction/apidoc/python/symbol.html).
We'll use Range Coding here since it has better compression performance.
At the end of this notebook you'll find an empirical comparison to Huffman Coding (which will turn out to perform worse).

### How to Implement Iterative Model Updates

So how are we going to perform these iterative model updates described above in practice?
Let's just start from how it's done in the the implementation of the model (which you cloned into the subdirectory `char-rnn.pytorch` in Step 3 above) and adapt it to our needs.
The file `generate.py` seems like a good place to look for inspiration since generating random text from the model is not all that different from what we're trying to do: both have to update the model character by character, continuously obtaining a probability distribution over the next character.
The only difference is that `generate.py` uses the obtained probability distribution to sample from it while we'll use it to define an entropy model.

The function `generate` in `char-rnn.pytorch/generate.py` shows us how to do all the steps we need.
The function takes a model as argument, which is—confusingly—called `decoder`.
The function body starts by initializing a hidden state as follows,

```python
    hidden = decoder.init_hidden(1)
```

After some more initializations, the function enters a loop that iteratively updates the model as follows:

```python
    output, hidden = decoder(inp, hidden)
```

where `inp` is the "input", i.e., the previous character, represented (for technical reasons) as an integer PyTorch tensor of length `1`.
The above model update returns `output` and the updated `hidden` state.
On the next line of code, `output` gets scaled by some temperature and exponentiated before it is interpreted as an (unnormalized) probability distribution by being passed into `torch.multinomial`.
Thus, `output` seems to be a tensor of logits, defining the probability distribution for the next character.
Finally, after drawing a sample `top_i` from the `torch.multinomial` distribution, the function maps the sampled integer to a character as follows:

```python
    predicted_char = all_characters[top_i]
```

Thus, there seems to be some fixed string `all_characters` that has the character with integer representation `i` at its `i`'th position.
Let's bring this string into scope (and while we're at it, let's also import some other stuff we'll need below):

In [5]:
import constriction
import torch
import numpy as np
from tqdm import tqdm

# Add subdirectory `char-rnn.pytorch` to Python's path so we can import some stuff from it.
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "char-rnn.pytorch"))
from model import *
from helpers import read_file, all_characters

Try it out:

In [6]:
print(all_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	



### Implement The Encoder

We now have everything we need to know to implement the encoder and decoder.
Let's start with the encoder, and define a function `compress_file` that takes the path to a file, compresses it, and writes the output to a different file.
Since our compression method will turn out to be very slow (see introduction), we'll also introduce an optional argument `max_chars` that will allow the caller to compress only the first `max_chars` characters from the input file and stop after that.
The function `compress_file` will also expect a `model` argument where the caller will have to pass in the trained model (this was called `decoder` in the file `generate.py` discussed above, but we'll call it `model` here to avoid confusion).

In [7]:
def compress_file(model, in_filename, out_filename, max_chars=None):
    message, _ = read_file(in_filename) # (`read_file` defined in `char-rnn.pytorch/helpers.py`)
    if max_chars is not None:
        message = message[:max_chars]   # Truncate message to at most `max_chars` characters.

    # Initialize the hidden state and model input as discussed above:
    hidden = model.init_hidden(1) # (same as in `generate.py` discussed above)
    input_char = torch.tensor([all_characters.index('\n')], dtype=torch.int64) # "fake" character that we pretend precedes the message

    # Instantiate an empty Range Coder onto which we'll accumulate compressed data:
    encoder = constriction.stream.queue.RangeEncoder()

    # Iterate over the message and encode it character by character, updating the model as we go:
    for char in tqdm(message):
        output, hidden = model(input_char, hidden) # update the model (as in `generate.py`)

        # Turn the `output` into an entropy model and encode the character with it:
        logits = output.data.view(-1)
        logits = logits - logits.max() # "Log-Sum-Exp trick" for numerical stability
        unnormalized_probs = logits.exp().numpy().astype(np.float64)
        entropy_model = constriction.stream.model.Categorical(unnormalized_probs)
        char_index = all_characters.index(char)
        encoder.encode(char_index, entropy_model)

        # Prepare for next model update:
        input_char[0] = char_index

    # Save the compressed data to a file
    print(f"Compressed {len(message)} characters into {encoder.num_bits()} bits ({encoder.num_bits() / len(message):.2f} bits per character).")
    compressed = encoder.get_compressed()
    if sys.byteorder != "little":
        # Let's always save data in the same byte order so compressed files can be transferred across computer architectures.
        compressed.byteswap(inplace=True)
    compressed.tofile(out_filename)
    print(f'Wrote compressed data to file "{out_filename}".')

The main part that distinguishes our function `compress_file` from the function `generate` in `generate.py` discussed above is that, after each model update, we use the model to encode an (already given) character rather than to sample a character.
There are two slightly subtle steps here:
first, we subtract the constant `logits.max()` from all elements of `logits` before exponentiating them.
Such a global shift in logit-space has no effect (apart from rounding errors) on the resulting probability distribution since it will correspond to a global scaling factor after exponentiation.
We perform this operation out of an abundance of caution to prevent numerical overflow when we exponentiate `logits` on the next line.
Second, we construct the `Categorical` entropy model from a tensor of *unnormalized* probabilities.
That's OK according to [the documentation](https://bamler-lab.github.io/constriction/apidoc/python/stream/model.html#constriction.stream.model.Categorical), `constriction` will have to make sure the distribution is exactly normalized in fixed-point arithmetic anyway.

### Implement The Decoder

Let's also implement a function `decompress_file` that recovers the message from its compressed representation so that we can prove that the encoder did not discard any information.
The decoder operates very similar to the encoder, except that it starts by loading compressed data from a file instead of the message, and it initializes a `RangeDecoder` from it, from which it then decodes one symbol at a time in the iteration.
One important difference to the encoder is that, with our current autoregressive model, the decoder cannot detect the end of the message.
Therefore, we have to provide the message length (`num_chars`) as an argument to the decoder function.
In a real deployment, you'll probably want to either transmit the message length as an explicit initial symbol, or you could add an "End of File" sentinel symbol to the alphabet (`all_characters`) and append this symbol to the message on the encoder side to signal to the decoder that it should stop processing.

In [8]:
def decompress_file(model, in_filename, out_filename, num_chars):
    # Load the compressed data into a `RangeDecoder`:`
    compressed = np.fromfile(in_filename, dtype=np.uint32)
    if sys.byteorder != "little":
        compressed.byteswap(inplace=True) # restores native byte order ("endianness").
    print(f"Loaded {32 * len(compressed)} bits of compressed data.")
    decoder = constriction.stream.queue.RangeDecoder(compressed)

    # Initialize the hidden state and model input exactly as in the encoder:
    hidden = model.init_hidden(1) # (same as in `generate.py` discussed above)
    input_char = torch.tensor([all_characters.index('\n')], dtype=torch.int64) # "fake" character that we pretend precedes the message

    # Decode the message character by character, updating the model as we go:
    with open(out_filename, "w") as out_file:
        for _ in tqdm(range(num_chars)):
            # Update model and obtain (unnormalized) probabilities, exactly as in the encoder:
            output, hidden = model(input_char, hidden)
            logits = output.data.view(-1)
            logits = logits - logits.max()
            unnormalized_probs = logits.exp().numpy().astype(np.float64)
            entropy_model = constriction.stream.model.Categorical(unnormalized_probs)
            
            # This time, use the `entropy_model` for *decoding* to obtain the next character:
            char_index = decoder.decode(entropy_model)
            char = all_characters[char_index]
            out_file.write(char)

            # Prepare for next model update, exactly as in the encoder:
            input_char[0] = char_index

    print(f'Wrote decompressed data to file "{out_filename}".')

## Step 5: Try It Out

If you've followed along and taken the time to understand the encoder/decoder implementation in Step 4 above then the model should have finished training by now.
Load it into memory:

In [9]:
model = torch.load("shakespeare_train.pt")

Now, try out if our implementation can indeed compress and decompress a text file with this model.
We'll compress the *test* subset of our data set so that we test our method on text that was not used for training (albeit, admittedly, the test data is very similar to the training data since both were written by the same author):

In [10]:
compress_file(model, "shakespeare_test.txt", "shakespeare_test.txt.compressed", max_chars=10_000)

100%|██████████| 10000/10000 [00:08<00:00, 1205.50it/s]


Compressed 10000 characters into 20640 bits (2.06 bits per character).
Wrote compressed data to file "shakespeare_test.txt.compressed".


If you didn't change anything in the training schedule then you should get a bitrate of about 2.1 bits per character.
Before we compare this bitrate to that of general-purpose compression methods, let's first verify that the compression method is actually correct.
Decode the compressed file again:

In [11]:
decompress_file(model, "shakespeare_test.txt.compressed", "shakespeare_test.txt.decompressed", 10_000)

Loaded 20640 bits of compressed data.


100%|██████████| 10000/10000 [00:07<00:00, 1275.40it/s]


Wrote decompressed data to file "shakespeare_test.txt.decompressed".


Let's take a quick peak in the original and reconstructed text:

In [12]:
!head shakespeare_test.txt shakespeare_test.txt.decompressed

==> shakespeare_test.txt <==
Before we proceed any further, hear me speak.
All:
First Citizen:
We are accounted poor citizens, the patricians good.
would yield us but the superfluity, while it were
but they think we are too dear: the leanness that
speak this in hunger for bread, not in thirst for revenge.
report fort, but that he pays himself with being proud.
First Citizen:
Come, come.

==> shakespeare_test.txt.decompressed <==
Before we proceed any further, hear me speak.
All:
First Citizen:
We are accounted poor citizens, the patricians good.
would yield us but the superfluity, while it were
but they think we are too dear: the leanness that
speak this in hunger for bread, not in thirst for revenge.
report fort, but that he pays himself with being proud.
First Citizen:
Come, come.


The beginnings certainly look similar.
But let's check more thoroughly.
Remember that we only encoded and decoded the first 10,000 characters of the test data, so that's what we have to compare to (note: the test turns out to be pure ASCII, so the first 10,000 characters map exactly to the first 10,000 bytes):


In [13]:
!head -c 10000 shakespeare_test.txt | diff - shakespeare_test.txt.decompressed  # If this prints no output we're good.

## Step 6: Evaluation

There's a lot we could analyze now:

- How do the bitrates of our method compare to general-purpose compression methods like `gzip`, `bzip2`, and `xz`?
- How well does our method generalize to other text, ranging from other English text by a different author all the way to text in a different language?
- Where do the encoder and decoder spend most of their runtime?

We'll just address the first question here and leave the others to the reader.
Let's compress the same first 10,000 characters of the test data with `gzip`, `bzip2`, and `xz` (if installed on your system):

In [14]:
!head -c 10000 shakespeare_test.txt | gzip --best > shakespeare_test.txt.gzip
!head -c 10000 shakespeare_test.txt | bzip2 --best > shakespeare_test.txt.bz2
!head -c 10000 shakespeare_test.txt | xz --best > shakespeare_test.txt.xz

Then let's compare the sizes of the compressed files:

In [15]:
!ls -l shakespeare_test.txt.*

-rw-rw-r-- 1 robamler robamler  4302 Jan  8 21:51 shakespeare_test.txt.bz2
-rw-rw-r-- 1 robamler robamler  2580 Jan  8 21:50 shakespeare_test.txt.compressed
-rw-rw-r-- 1 robamler robamler 10000 Jan  8 21:50 shakespeare_test.txt.decompressed
-rw-rw-r-- 1 robamler robamler  4814 Jan  8 21:51 shakespeare_test.txt.gzip
-rw-rw-r-- 1 robamler robamler  4788 Jan  8 21:51 shakespeare_test.txt.xz


Despite using a very simple model that we took from a tutorial completely unrelated to data compression, our compression method reduces the bitrate compared to `bzip2` by 40%.
Of course, we shouldn't read too much into this since we trained the model on data that is very similar to the test data.

## Bonus 1: Getting It *Almost* Right and Yet Fatally Wrong

The [API reference for `constriction`'s entropy models](https://bamler-lab.github.io/constriction/apidoc/python/stream/model.html) highlights that entropy models are brittle: event tiny discrepancies in rounding operations between encoder and decoder can have catastrophic effects for entropy coding.
The models provided by `constriction` are implemented in exact fixed point arithmetic to allow for well-defined and consistent rounding operations when, e.g., inverting the CDF.
However, `constriction` can only guarantee consistent rounding operations in its internal operations.
You have to ensure yourself that any probabilities you provide to `constriction` are *exactly* the same on the encoder and decoder side.

The following example illustrates how even tiny discrepancies between rounding operations on the encoder and decoder side can completely derail the entropy coder.
Recall that, in both functions `compress_file` and `decompress_file` above, we subtract `logits.max()` from `logits` before we exponentiate them.
This can prevent numerical overflow in the exponentiation but one might expect that it has otherwise no effect on the resulting probability distribution since it only leads to a global scaling of `unnormalized_probs`, which should drop out once `constriction` normalizes the probabilities—*except* that this is not quite correct:
even if there's no numerical overflow, the different scaling affects all subsequent rounding operations that are implicitly performed by the CPU in any floating point operation.
In and of itself, these implicit rounding operations are not a big issue and unavoidable in floating point calculations.
However, they do become an issue when they are done inconsistently between the encoder and the decoder, as we show in the next example.

Let's keep the encoder as it is, but let's slightly modify the decoder by commenting out the line that subtracts `logits.max()` from `logits`, as highlighted by the string `<--- COMMENTED OUT` in the following example:

In [16]:
def decompress_file_almost_right(model, in_filename, out_filename, num_chars):
    # Load the compressed data into a `RangeDecoder`:`
    compressed = np.fromfile(in_filename, dtype=np.uint32)
    if sys.byteorder != "little":
        compressed.byteswap(inplace=True) # restores native byte order ("endianness").
    print(f"Loaded {32 * len(compressed)} bits of compressed data.")
    decoder = constriction.stream.queue.RangeDecoder(compressed)

    # Initialize the hidden state and model input exactly as in the encoder:
    hidden = model.init_hidden(1) # (same as in `generate.py` discussed above)
    input_char = torch.tensor([all_characters.index('\n')], dtype=torch.int64) # "fake" character that we pretend precedes the message

    # Decode the message character by character, updating the model as we go:
    with open(out_filename, "w") as out_file:
        for _ in tqdm(range(num_chars)):
            # Update model and optain (unnormalized) probabilities, exactly as in the encoder:
            output, hidden = model(input_char, hidden)
            logits = output.data.view(-1)
            # logits = logits - logits.max()                               <--- COMMENTED OUT
            unnormalized_probs = logits.exp().numpy().astype(np.float64)
            entropy_model = constriction.stream.model.Categorical(unnormalized_probs)
            
            # This time, use the `entropy_model` for *decoding* to obtain the next character:
            char_index = decoder.decode(entropy_model)
            char = all_characters[char_index]
            out_file.write(char)

            # Prepare for next model update, exactly as in the encoder:
            input_char[0] = char_index

    print(f'Wrote decompressed data to file "{out_filename}".')

If we use this slightly modified decoder to decode data that was encoded with the original encoder `compress_file`, we *might* run into an issue.

**Note:** The following example may or may not work on your setup, depending on the random seeds used for training the model.
But if it fails, as in my setup below, then it will fail catastrophically and either throw an error (as it does here) or (if we're not quite as lucky) silently continue decoding but decode complete gibberish after some point in the stream.

In [17]:
decompress_file_almost_right(model, "shakespeare_test.txt.compressed", "shakespeare_test.txt.decompressed_wrong", 10_000)

Loaded 20640 bits of compressed data.


  9%|▉         | 884/10000 [00:00<00:08, 1110.52it/s]


AssertionError: Tried to decode from compressed data that is invalid for the employed entropy model.

Notice that we were able to decode the first 883 characters just fine (you might get a different number here).
Issues due to tiny discrepancies in rounding operations are very unlikely to occur, but when they occur in an entropy coder, they are fatal.
We actually got lucky here: it's also possible that the decoder does not detect any errors but that it starts decoding complete gibberish after some point (in fact, had we used an ANS coder instead of a Range Coder, then decoding would have been infallible but could still produce wrong results when misused).

Due to this brittleness, entropy models have to be implemented with care, and we consider `constriction`'s implementations of entropy models an important part of the library, in addition to `constriction`'s entropy coders.
Yet, as the above example shows, even the most careful implementation of an entropy model cannot protect from errors when misused.

## Bonus 2: Huffman Coding

Our above compression method uses Range Coding for the entropy coder.
The `constriction` library also provides another entropy coder with "queue" semantics: Huffman coding.
The API for Huffman coding is somewhat different to that of Range Coding because of the very different nature of the two algorithms, but it's easy to port our encoder and decoder to Huffman coding:

### Encoder

In [18]:
def compress_file_huffman(model, in_filename, out_filename, max_chars=None):
    message, _ = read_file(in_filename) # (`read_file` defined in `char-rnn.pytorch/helpers.py`)
    if max_chars is not None:
        message = message[:max_chars]   # Truncate message to at most `max_chars` characters.

    # Initialize the hidden state and model input as discussed above:
    hidden = model.init_hidden(1) # (same as in `generate.py` discussed above)
    input_char = torch.tensor([all_characters.index('\n')], dtype=torch.int64) # "fake" character that we pretend precedes the message

    # Instantiate an empty `QueueEncoder` onto which we'll accumulate compressed data:
    encoder = constriction.symbol.QueueEncoder()                                       # <-- CHANGED LINE

    # Iterate over the message and encode it character by character, updating the model as we go:
    for char in tqdm(message):
        output, hidden = model(input_char, hidden) # update the model (as in `generate.py`)

        # Turn the `output` into an entropy model and encode the character with it:
        logits = output.data.view(-1)
        logits = logits - logits.max() # "Log-Sum-Exp trick" for numerical stability
        unnormalized_probs = logits.exp().numpy().astype(np.float64)
        codebook = constriction.symbol.huffman.EncoderHuffmanTree(unnormalized_probs)  # <-- CHANGED LINE
        char_index = all_characters.index(char)
        encoder.encode_symbol(char_index, codebook)                                    # <-- CHANGED LINE

        # Prepare for next model update:
        input_char[0] = char_index

    # Save the compressed data to a file
    compressed, num_bits = encoder.get_compressed()                                    # <-- CHANGED LINE
    print(f"Compressed {len(message)} characters into {num_bits} bits ({num_bits / len(message):.2f} bits per character).")
    if sys.byteorder != "little":
        # Let's always save data in the same byte order so compressed files can be transferred across computer architectures.
        compressed.byteswap(inplace=True)
    compressed.tofile(out_filename)
    print(f'Wrote compressed data to file "{out_filename}".')

### Decoder

In [19]:
def decompress_file_huffman(model, in_filename, out_filename, num_chars):
    # Load the compressed data into a `RangeDecoder`:`
    compressed = np.fromfile(in_filename, dtype=np.uint32)
    if sys.byteorder != "little":
        compressed.byteswap(inplace=True) # restores native byte order ("endianness").
    print(f"Loaded {32 * len(compressed)} bits of compressed data.")
    decoder = constriction.symbol.QueueDecoder(compressed)                                 # <-- CHANGED LINE

    # Initialize the hidden state and model input exactly as in the encoder:
    hidden = model.init_hidden(1) # (same as in `generate.py` discussed above)
    input_char = torch.tensor([all_characters.index('\n')], dtype=torch.int64) # "fake" character that we pretend precedes the message

    # Decode the message character by character, updating the model as we go:
    with open(out_filename, "w") as out_file:
        for _ in tqdm(range(num_chars)):
            # Update model and optain (unnormalized) probabilities, exactly as in the encoder:
            output, hidden = model(input_char, hidden)
            logits = output.data.view(-1)
            logits = logits - logits.max()
            unnormalized_probs = logits.exp().numpy().astype(np.float64)
            codebook = constriction.symbol.huffman.DecoderHuffmanTree(unnormalized_probs)  # <-- CHANGED LINE
            
            # This time, use the `codebook` for *decoding* to obtain the next character:
            char_index = decoder.decode_symbol(codebook)                                   # <-- CHANGED LINE
            char = all_characters[char_index]
            out_file.write(char)

            # Prepare for next model update, exactly as in the encoder:
            input_char[0] = char_index

    print(f'Wrote decompressed data to file "{out_filename}".')

### Try it Out Again

In [20]:
compress_file_huffman(model, "shakespeare_test.txt", "shakespeare_test.txt.compressed-huffman", max_chars=10_000)

100%|██████████| 10000/10000 [00:05<00:00, 1689.45it/s]


Compressed 10000 characters into 23278 bits (2.33 bits per character).
Wrote compressed data to file "shakespeare_test.txt.compressed-huffman".


For comparison: when we used a Range Coder we got a better bitrate of only about 2.1 bits per character.

In [21]:
decompress_file_huffman(model, "shakespeare_test.txt.compressed-huffman", "shakespeare_test.txt.decompressed-huffman", 10_000)

Loaded 23296 bits of compressed data.


100%|██████████| 10000/10000 [00:07<00:00, 1358.89it/s]


Wrote decompressed data to file "shakespeare_test.txt.decompressed-huffman".


Verify correctness again:

In [22]:
!head -c 10000 shakespeare_test.txt | diff - shakespeare_test.txt.decompressed-huffman  # If this prints no output we're good.

## Conclusion

We've discussed how you can use `constriction`'s entropy coders with an entropy model that is an autoregressive machine-learning model.
Autoregressive models allow you to model correlations between symbols, and exploit them to improve compression performance.
An alternative method for exploiting correlations in data compression is the so-called bits-back technique, which applies to latent variable models that tend to be better parallelizable than autoregressive models.
An example of bits-back coding with `constriction` is provided in [this problem set](https://robamler.github.io/teaching/compress21/problem-set-05.zip) (with [solutions](https://robamler.github.io/teaching/compress21/problem-set-05-solutions.zip)).